# Basic QUBO HUBO solver

In [31]:
##### Initial setting #####
# x1 = q0 + 2q1 + 4q2 + --- + 2^(n-1)q(n-1)
# x2 = qn + 2q(n+1) + 4q(n+2) + --- 2^(n-1)q(2n-1)
# x1x2 = c
# Least square problem
# QUBO = (x1x2 - c)^2 - c^2
import numpy as np
from dwave.system import DWaveSampler, EmbeddingComposite
import dimod
x1 = int(3)
x2 = int(5)
c = x1 * x2
print("first prime number: ", x1)
print("second prime number: ", x2)
print("RSA number: ", c)

first prime number:  3
second prime number:  5
RSA number:  15


## Factorization problem to HUBO

In [32]:
qubits = 3
max_d = format(len(str(2 * qubits)), '02')
QM = np. zeros((2 * qubits, 2 * qubits))
Q = {}

# Linear term
for j in range(qubits):
    for i in range(qubits):
        po1 = i
        po2 = qubits + j
        val = pow(2, 2 * (i + j)) - 2 * c * pow(2, i + j)
        exec("Q.update({('q%s','q%s'):%s})" % (
            format(po1 + 1, max_d), format(po2 + 1, max_d), format(val)))

# Quadratic term
for k in range(qubits):
    for i in range(qubits - 1):
        for j in range(i + 1, qubits):
            # 2^(i+j+2k+1)aiajbk
            po1 = i
            po2 = j
            po3 = qubits + k
            val = pow(2, i + j + 2 * k + 1)
            exec("Q.update({('q%s','q%s','q%s'):%s})" % (format(po1 + 1, max_d), format(po2 + 1, max_d),
                                                         format(po3 + 1, max_d), format(val)))

# Cubic term
for k in range(qubits):
    for i in range(qubits - 1):
        for j in range(i + 1, qubits):
            # 2^(i+j+2k+1)akbibj
            po1 = k
            po2 = qubits + i
            po3 = qubits + j
            val = pow(2, i + j + 2 * k + 1)
            exec("Q.update({('q%s','q%s','q%s'):%s})" % (format(po1 + 1, max_d), format(po2 + 1, max_d),
                                                         format(po3 + 1, max_d), format(val)))

# Quartic term
for i2 in range(qubits - 1):
    for j2 in range(i2 + 1, qubits):
        for i1 in range(qubits - 1):
            for j1 in range(i1 + 1, qubits):
                po1 = i1
                po2 = j1
                po3 = qubits + i2
                po4 = qubits + j2
                val = pow(2, i1 + j1 + i2 + j2 + 2)
                exec("Q.update({('q%s','q%s','q%s','q%s'):%s})" % (format(po1 + 1, max_d), format(po2 + 1, max_d),
                                                                   format(po3 + 1, max_d), format(po4 + 1, max_d), format(val)))
                


## HUBO to QUBO and solving

In [1]:
sampler = DWaveSampler()
sampler_auto = EmbeddingComposite(sampler)
print("Connected to sampler", sampler.solver.name)

# sampleset = dimod.ExactPolySolver().sample_hubo(Q)
# hubo = dimod.higherorder.BinaryPolynomial.from_hubo(Q)

# ising = dimod.higherorder.BinaryPolynomial.to_binary(hubo)
print(Q)

qubo = dimod.make_quadratic(Q, 5, dimod.BINARY)

print(qubo)

sampleset = dimod.ExactSolver().sample_qubo(qubo)

# energy = 0
# energies = sampleset.record.energy
# energy0_nums = np. where(energies == - pow(c, 2))[0]
# x = np. zeros(2)
# for idx in range(len(energy0_nums)):
#     sol1 = sampleset. record[energy0_nums[idx]][0]
#     for xk in range(2):
#         x[xk] = 0
#     lambda1 = 0
#     for xk in range(2):
#         for k in range(qubits):
#             x[xk] = x[xk] + pow(2, k) * (sol1[xk * qubits + k])
#     print(x)


NameError: name 'DWaveSampler' is not defined